# Q_ RATE OF CHANGE

In [1]:
from koselleck import *

In [2]:
def do_measure_change_over_years(obj,words=None,progress=True,k=50,**y):
    try:
        row1,row2=obj
        if words is None: words=get_valid_words()
        m1,m2=load_model_row(row1),load_model_row(row2)
#         odf=measure_change(m1,m2,words=words,progress=progress,
# #                            funcs=[measure_change_procrustes],
#                            funcs=[measure_change_noiseaware],
#                            **y).reset_index()
#         odf=measure_change_noiseaware(m1,m2,words=words,progress=progress,**y)
#         odf=measure_change_procrustes(m1,m2,words=words,progress=progress,**y)
        odf=measure_change_local(m1,m2,words=words,k=k,progress=progress,**y)
        odf['period1'],odf['period2']=row1.period,row2.period
        return odf
    except Exception:
        return pd.DataFrame()

def measure_change_over_years(dfgrp,words=None,num_proc=1,progress=True,**y):
    dfgrp=dfgrp.sort_values('period_start')
    row1=None
    m1=None
    odf=pd.DataFrame()
    #iterr=tqdm(dfgrp.iterrows(),total=len(dfgrp)) if progress else dfgrp.iterrows()
    rows=[row for i,row in dfgrp.iterrows()]
    objs=[
        (row1,row2)
        for i1,row1 in dfgrp.iterrows() 
        for i2,row2 in dfgrp.iterrows() 
        if row2.period>row1.period
    ]
#     objs=random.sample(objs,3)
    resdfs=[
        x for x in pmap(
            do_measure_change_over_years,
            objs,
            num_proc=num_proc,
            progress=progress,
            kwargs=dict(
                progress=False,
                words=words,
                **y
            ),
            desc='Measuring rate of change across years'
        ) if type(x)==pd.DataFrame
    ]
    return pd.concat(resdfs) if len(resdfs) else pd.DataFrame()

In [ ]:
dfdecs=get_pathdf_models().query('period_len==5 & run=="run_01" & period_start>=1720 & period_start<1900')
grps=dfdecs.groupby('run')
for i,dfgrp in grps: pass
# dfgrp

In [ ]:
# gen
# odf=measure_change_over_years(dfgrp.iloc[:3],num_proc=4)
odf=measure_change_over_years(dfgrp,num_proc=4)
# save
odf.to_pickle('data/data.semantic_change_over_decades.1run.v10-local-k50-halfdec.pkl')
odf

Measuring rate of change across years [x4]:  29%|██▊       | 180/630 [49:58<2:18:35, 18.48s/it]